## Individual run

In [1]:
import pickle

In [2]:
import sys
sys.path.append('../')
from util import corpus, tokenize, util, alignment, word2vec, retrieval

[nltk_data] Downloading package stopwords to /home/sudhi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sudhi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# df = util.decompress_pickle('../../data/word2vec/epo/corpus/epo_18_20_df_phrasal')
df = util.decompress_pickle('../../data/epo_corpus_all_years/pre_proc/epo_10_20_df_w_ncs')
df

In [4]:
df.en[0]

'A modular power converter comprising : a thermal_support NUM for a circuit , the support comprising a substrate having fluid_inlet NUM and outlet ports NUM and a circulation_path NUM coupled between the inlet and outlet ports , the inlet and outlet ports being configured to transmit a coolant for circulation through the circulation_path ; an interface_plate NUM configured to support a power_electronics_circuit NUM , the interface_plate being configured for mounting to the support adjacent to the coolant circulation_path for extraction of heat from the interface_plate during operation ; and a power_electronics_circuit configured for power_conversion supported on the interface_plate ; and a driver_circuit NUM for applying the drive_signals to the power_electronics_circuit , the driver_circuit being secured to and cooled by the thermal_support ; wherein the support at least partially defines an electrical_reference_plane , a mechanical_support , and a thermal_extraction_path for the powe

In [5]:
epo_corpus = corpus.Corpus(name='epo', 
                           doc=df,
                           tokenize_corpus=True,
                           load_vocab=True,
                           load_tokens=True)

19:39:27: Loading vocab
19:39:28: Loading tokens
19:39:37: Artefacts in the path: ../../data/word2vec/epo/
19:39:37: Sentences
	Lang	Size
	EN	365024
	DE	365024
19:39:37: Vocab
	Lang	Size
	EN	380934
	DE	552476


In [6]:
algn_dic = alignment.AlignmentDic(name='epo',
                                 corpus=epo_corpus,
                                 rerun=True)

19:39:38: Fetching alignment dictionary from path ../../data/word2vec/epo/


In [7]:
algn_dic.load_trans_dic()

19:39:40: Loading translation dictionary from path: ../../data/word2vec/epo/alg/transidx


In [10]:
len(algn_dic.idx2transidx.keys())

190460

In [44]:
len(algn_dic.idx2transidx), len(epo_corpus.idx2word)

(190460, 933410)

In [8]:
epo_corpus.idx2word[algn_dic.idx2transidx[epo_corpus.word2idx['vehicle']]]

'fahrzeug'

In [33]:
from imp import reload
reload(word2vec)

<module 'util.word2vec' from '../util/word2vec.py'>

In [34]:
w2v = word2vec.Word2Vec(name='epo', 
                        corpus=epo_corpus,
                        algn_dic=algn_dic,
                        rerun_data=True,
                        rerun_train=True,
                        remove_underscores=True
                       )


# w2v = word2vec.Word2Vec(name='epo', 
#                         corpus=epo_corpus,
#                         algn_dic=algn_dic,
#                         rerun_data=False,
#                         rerun_train=False,
#                        )

21:01:31: Fetching contents from path ../../data/word2vec/epo/


In [35]:
w2v.generate_train_data()

21:01:33: Generating train data


21:02:40: Saved train data to path: ../../data/word2vec/epo/train/aligned_sentences


In [23]:
w2v.load_train_data()

20:08:30: Loading train data from path: ../../data/word2vec/epo/train/aligned_sentences


In [24]:
len(w2v.aligned_sentences)

2980261

In [36]:
len(w2v.aligned_sentences)

3136108

In [37]:
from tqdm.notebook import tqdm as tq

wo_ncs_sent = []
for sent in tq(w2v.aligned_sentences):
    wo_ncs = []
    for token in sent:
        s = token.split('_')
        for _ in s:
            wo_ncs.append(_)
    wo_ncs_sent.append(wo_ncs)
# print(wo_ncs_sent)

In [38]:
len(wo_ncs_sent)

3136108

In [39]:
train_sent = wo_ncs_sent + w2v.aligned_sentences
len(train_sent)

6272216

In [ ]:
w2v.train_model(vector_size=300, window=5, min_count=1)

In [41]:
w2v.build_wv_vocab()

21:43:13: Building Word2Vec vocab


In [42]:
len(w2v.wv_vocab)

339588

## Projection

In [19]:
from util import projection

In [20]:
projection.generate_tsv_files(w2v)


Vectors saved at: vectors_epo.tsv
Words saved at: words_epo.tsv
